In [1]:
import altair as alt
from altair.expr import datum, if_
import datetime
import json
import numpy as np
import pandas as pd
import psycopg2
pd.options.display.float_format = "{:,.2f}".format

Connect to the database.

In [2]:
with open("config.json") as f:
    conf = json.load(f)
conn = psycopg2.connect(
    dbname=conf["database"],
    user=conf["user"],
    host=conf["host"],
    password=conf["password"]
)
conn.autocommit = True

## Return the donors who gave at least $50,000 to conservative and GOP-aligned groups between Nov. 2 and Dec. 31, 2017, split by their giving before the tax bill's introduction on Nov. 2 and after its introduction. Do this for every off-year going back to the 2010 cycle.

In [3]:
def run_donors_query(prefix, suffix, reference_cycle, reference_start, reference_end, cycle, start, end):
    query = """CREATE MATERIALIZED VIEW IF NOT EXISTS """+prefix+"""_donors_committees_"""+suffix+""" AS
                    SELECT match_id,
                           organizations,
                           contributors,
                           sum(committee_total) AS total,
                           cmte_id,
                           pacshort AS committee
                    FROM
                      (SELECT match_id,
                              organizations,
                              contributors,
                              sum(amount) AS committee_total,
                              cmteid AS cmte_id
                       FROM
                         (SELECT CASE
                                     WHEN trim(contribid) != '' THEN left(contribid, 11)
                                     ELSE orgname
                                 END AS match_id,
                                 array_agg(DISTINCT orgname) AS organizations,
                                 array_agg(DISTINCT contrib) AS contributors
                          FROM crp_contributions
                          LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
                          AND crp_committees.cycle = '""" + reference_cycle + """'
                          WHERE primcode IN ('J1100',
                                             'J2200',
                                             'J2400',
                                             'Z1100',
                                             'Z4100',
                                             'Z4500',
                                             'Z5100')
                            AND date >= '""" + reference_start + """'
                            AND date <= '""" + reference_end + """'
                            AND crp_contributions.cycle = '""" + reference_cycle + """'
                            AND TYPE NOT IN ('11J',
                                             '15C',
                                             '15E',
                                             '15I',
                                             '15J',
                                             '15T',
                                             '18J',
                                             '19J',
                                             '30J',
                                             '30F',
                                             '31J',
                                             '31F',
                                             '32J',
                                             '32F')
                          GROUP BY match_id
                          HAVING sum(amount) >= 50000) AS gop_donors
                       JOIN crp_contributions ON CASE
                                                     WHEN trim(contribid) != '' THEN left(contribid, 11)
                                                     ELSE crp_contributions.orgname
                                                 END = gop_donors.match_id
                       WHERE date >= '""" + start + """'
                         AND date <= '""" + end + """'
                         AND CYCLE = '""" + cycle + """'
                         AND TYPE NOT IN ('11J',
                                          '15C',
                                          '15E',
                                          '15I',
                                          '15J',
                                          '15T',
                                          '18J',
                                          '19J',
                                          '30J',
                                          '30F',
                                          '31J',
                                          '31F',
                                          '32J',
                                          '32F')
                       GROUP BY match_id,
                                organizations,
                                contributors,
                                cmte_id) AS donors_committees
                    JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
                    AND crp_committees.CYCLE = '""" + cycle + """'
                    WHERE primcode IN ('J1100',
                                       'J2200',
                                       'J2400',
                                       'Z1100',
                                       'Z4100',
                                       'Z4500',
                                       'Z5100')
                    GROUP BY match_id,
                             organizations,
                             contributors,
                             cmte_id,
                             committee;

                    SELECT *
                    FROM """+prefix+"""_donors_committees_"""+suffix+""";
                    """
    return pd.read_sql(query, con=conn)

## Return the data for each cycle.

In [4]:
post_bill_donors_committees_17 = run_donors_query("post_bill", "17", "2018", "2017-11-02", "2017-12-31", "2018", "2017-11-02", "2017-12-31")
pre_bill_donors_committees_17 = run_donors_query("pre_bill", "17", "2018", "2017-11-02", "2017-12-31", "2018", "2017-01-01", "2017-11-01")

In [5]:
post_bill_donors_committees_15 = run_donors_query("post_bill", "15", "2018", "2017-11-02", "2017-12-31", "2016", "2015-11-02", "2015-12-31")
pre_bill_donors_committees_15 = run_donors_query("pre_bill", "15", "2018", "2017-11-02", "2017-12-31", "2016", "2015-01-01", "2015-11-01")

In [6]:
post_bill_donors_committees_13 = run_donors_query("post_bill", "13", "2018", "2017-11-02", "2017-12-31", "2014", "2013-11-02", "2013-12-31")
pre_bill_donors_committees_13 = run_donors_query("pre_bill", "13", "2018", "2017-11-02", "2017-12-31", "2014", "2013-01-01", "2013-11-01")

In [7]:
post_bill_donors_committees_11 = run_donors_query("post_bill", "11", "2018", "2017-11-02", "2017-12-31", "2012", "2011-11-02", "2011-12-31")
pre_bill_donors_committees_11 = run_donors_query("pre_bill", "11", "2018", "2017-11-02", "2017-12-31", "2012", "2011-01-01", "2011-11-01")

In [8]:
post_bill_donors_committees_09 = run_donors_query("post_bill", "09", "2018", "2017-11-02", "2017-12-31", "2010", "2009-11-02", "2009-12-31")
pre_bill_donors_committees_09 = run_donors_query("pre_bill", "09", "2018", "2017-11-02", "2017-12-31", "2010", "2009-01-01", "2009-11-01")

## How much did each contributor give in each period for each cycle?

### 2017

In [9]:
post_bill_donors_17 = post_bill_donors_committees_17.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
pre_bill_donors_17 = pre_bill_donors_committees_17.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()

In [10]:
donors_17 = post_bill_donors_17.merge(pre_bill_donors_17, how="outer", on="match_id", suffixes=["_post_bill", "_pre_bill"])
donors_17.drop(["organizations_pre_bill", "contributors_pre_bill"], axis=1, inplace=True)
donors_17.rename(columns={"organizations_post_bill": "organizations", "contributors_post_bill": "contributors"}, inplace=True)
donors_17["total_pre_bill"].fillna(0, inplace=True)
donors_17.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158 entries, 0 to 157
Data columns (total 5 columns):
match_id           158 non-null object
organizations      158 non-null object
contributors       158 non-null object
total_post_bill    158 non-null float64
total_pre_bill     158 non-null float64
dtypes: float64(2), object(3)
memory usage: 7.4+ KB


In [11]:
donors_17["pct_post_bill"] = donors_17["total_post_bill"] / (donors_17["total_pre_bill"] + donors_17["total_post_bill"])
donors_17["pct_pre_bill"] = donors_17["total_pre_bill"] / (donors_17["total_pre_bill"] + donors_17["total_post_bill"])
donors_17["change"] = donors_17["total_post_bill"] - donors_17["total_pre_bill"]
donors_17["pct_change"] = (donors_17["total_post_bill"] - donors_17["total_pre_bill"]) / donors_17["total_pre_bill"].abs()
donors_17.sort_values("total_post_bill", ascending=False).head()

,match_id,organizations,contributors,total_post_bill,total_pre_bill,pct_post_bill,pct_pre_bill,change,pct_change
52,U0000003690,"[U-Line Corp, Uline Inc]","[UIHLEIN, ELIZABETH, UIHLEIN, ELIZABETH MRS, U...","4,557,300.00","12,051,400.00",0.27,0.73,"-7,494,100.00",-0.62
2,American Action Network,[American Action Network],[AMERICAN ACTION NETWORK],"3,747,520.00","9,142,525.00",0.29,0.71,"-5,395,005.00",-0.59
60,U0000004054,"[Ghpalmer Assoc, GH Palmer Assoc]","[PALMER, GEOFF, PALMER, GEOFFREY H]","1,955,200.00","546,272.00",0.78,0.22,"1,408,928.00",2.58
10,Hillwood Development,[Hillwood Development],[HILLWOOD DEVELOPMENT COMPANY LLC],"1,500,000.00","500,000.00",0.75,0.25,"1,000,000.00",2.00
67,U0000004552,"[Cinemark Holdings, Cinemark USA]","[MITCHELL, LEE, MITCHELL, LEE ROY, MITCHELL, T...","1,007,400.00","131,299.00",0.88,0.12,"876,101.00",6.67


#### How much did they give?

In [12]:
print("2017 post-bill total: $" + str(donors_17["total_post_bill"].sum()))
print("2017 pre-bill total: $" + str(donors_17["total_pre_bill"].sum()))

2017 post-bill total: $35639563.0
2017 pre-bill total: $58975282.0


### 2015

In [13]:
post_bill_donors_15 = post_bill_donors_committees_15.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
pre_bill_donors_15 = pre_bill_donors_committees_15.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()

In [14]:
donors_15 = post_bill_donors_15.merge(pre_bill_donors_15, how="outer", on="match_id", suffixes=["_post_bill", "_pre_bill"])
donors_15.drop(["organizations_pre_bill", "contributors_pre_bill"], axis=1, inplace=True)
donors_15.rename(columns={"organizations_post_bill": "organizations", "contributors_post_bill": "contributors"}, inplace=True)
donors_15["total_pre_bill"].fillna(0, inplace=True)
donors_15.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 111 entries, 0 to 110
Data columns (total 5 columns):
match_id           111 non-null object
organizations      80 non-null object
contributors       80 non-null object
total_post_bill    80 non-null float64
total_pre_bill     111 non-null float64
dtypes: float64(2), object(3)
memory usage: 5.2+ KB


In [15]:
donors_15["pct_post_bill"] = donors_15["total_post_bill"] / (donors_15["total_pre_bill"] + donors_15["total_post_bill"])
donors_15["pct_pre_bill"] = donors_15["total_pre_bill"] / (donors_15["total_pre_bill"] + donors_15["total_post_bill"])
donors_15["change"] = donors_15["total_post_bill"] - donors_15["total_pre_bill"]
donors_15["pct_change"] = (donors_15["total_post_bill"] - donors_15["total_pre_bill"]) / donors_15["total_pre_bill"].abs()
donors_15.sort_values("total_post_bill", ascending=False).head()

,match_id,organizations,contributors,total_post_bill,total_pre_bill,pct_post_bill,pct_pre_bill,change,pct_change
21,U0000003477,[Koch Industries],"[KOCH, CHARLES, KOCH, CHARLES G MR, KOCH, CHAR...","3,000,000.00",0.00,1.00,0.00,"3,000,000.00",inf
31,U0000003829,[Point72 Asset Management],"[COHEN, STEVE MR]","2,000,000.00","2,002,699.00",0.50,0.50,"-2,699.00",-0.00
4,Republican Governors Assn,[Republican Governors Assn],[REPUBLICAN GOVERNORS ASSOCIATION],"750,000.00","3,450,000.00",0.18,0.82,"-2,700,000.00",-0.78
27,U0000003682,[Renaissance Technologies],"[MERCER, DIANA, MERCER, DIANA MRS, MERCER, ROB...","560,500.00","14,517,100.00",0.04,0.96,"-13,956,600.00",-0.96
8,U0000000175,[Stephens Inc],"[STEPHENS, WARREN, STEPHENS, WARREN A MR]","549,400.00","3,485,123.00",0.14,0.86,"-2,935,723.00",-0.84


#### How much did they give?

In [16]:
print("2015 post-bill total: $" + str(donors_15["total_post_bill"].sum()))
print("2015 pre-bill total: $" + str(donors_15["total_pre_bill"].sum()))

2015 post-bill total: $1647560.0
2015 pre-bill total: $77617350.0


### 2013

In [17]:
post_bill_donors_13 = post_bill_donors_committees_13.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
pre_bill_donors_13 = pre_bill_donors_committees_13.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()

In [18]:
donors_13 = post_bill_donors_13.merge(pre_bill_donors_13, how="outer", on="match_id", suffixes=["_post_bill", "_pre_bill"])
donors_13.drop(["organizations_pre_bill", "contributors_pre_bill"], axis=1, inplace=True)
donors_13.rename(columns={"organizations_post_bill": "organizations", "contributors_post_bill": "contributors"}, inplace=True)
donors_13["total_pre_bill"].fillna(0, inplace=True)
donors_13.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109 entries, 0 to 108
Data columns (total 5 columns):
match_id           109 non-null object
organizations      62 non-null object
contributors       62 non-null object
total_post_bill    62 non-null float64
total_pre_bill     109 non-null float64
dtypes: float64(2), object(3)
memory usage: 5.1+ KB


In [19]:
donors_13["pct_post_bill"] = donors_13["total_post_bill"] / (donors_13["total_pre_bill"] + donors_13["total_post_bill"])
donors_13["pct_pre_bill"] = donors_13["total_pre_bill"] / (donors_13["total_pre_bill"] + donors_13["total_post_bill"])
donors_13["change"] = donors_13["total_post_bill"] - donors_13["total_pre_bill"]
donors_13["pct_change"] = (donors_13["total_post_bill"] - donors_13["total_pre_bill"]) / donors_13["total_pre_bill"].abs()
donors_13.sort_values("total_post_bill", ascending=False).head()

,match_id,organizations,contributors,total_post_bill,total_pre_bill,pct_post_bill,pct_pre_bill,change,pct_change
3,Republican Governors Assn,[Republican Governors Assn],[REPUBLICAN GOVERNORS ASSOCIATION],"1,000,000.00",0.00,1.00,0.00,"1,000,000.00",inf
25,U0000003903,[Alliance Coal],"[CRAFT, JOE W III, CRAFT, JOSEPH MR III, CRAFT...","515,600.00","162,900.00",0.76,0.24,"352,700.00",2.17
7,U0000000175,[Stephens Inc],"[STEPHENS, WARREN, STEPHENS, WARREN A MR]","442,400.00","51,300.00",0.90,0.10,"391,100.00",7.62
22,U0000003690,"[U-Line Corp, Uline Inc]","[UIHLEIN, ELIZABETH, UIHLEIN, ELIZABETH MRS, U...","261,400.00","533,150.00",0.33,0.67,"-271,750.00",-0.51
0,American Action Network,[American Action Network],[AMERICAN ACTION NETWORK],"109,714.00","81,237.00",0.57,0.43,"28,477.00",0.35


#### How much did they give?

In [20]:
print("2013 post-bill total: $" + str(donors_13["total_post_bill"].sum()))
print("2013 pre-bill total: $" + str(donors_13["total_pre_bill"].sum()))

2013 post-bill total: $3859464.0
2013 pre-bill total: $6716175.0


### 2011

In [21]:
post_bill_donors_11 = post_bill_donors_committees_11.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
pre_bill_donors_11 = pre_bill_donors_committees_11.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()

In [22]:
donors_11 = post_bill_donors_11.merge(pre_bill_donors_11, how="outer", on="match_id", suffixes=["_post_bill", "_pre_bill"])
donors_11.drop(["organizations_pre_bill", "contributors_pre_bill"], axis=1, inplace=True)
donors_11.rename(columns={"organizations_post_bill": "organizations", "contributors_post_bill": "contributors"}, inplace=True)
donors_11["total_pre_bill"].fillna(0, inplace=True)
donors_11.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 106 entries, 0 to 105
Data columns (total 5 columns):
match_id           106 non-null object
organizations      71 non-null object
contributors       71 non-null object
total_post_bill    71 non-null float64
total_pre_bill     106 non-null float64
dtypes: float64(2), object(3)
memory usage: 5.0+ KB


In [23]:
donors_11["pct_post_bill"] = donors_11["total_post_bill"] / (donors_11["total_pre_bill"] + donors_11["total_post_bill"])
donors_11["pct_pre_bill"] = donors_11["total_pre_bill"] / (donors_11["total_pre_bill"] + donors_11["total_post_bill"])
donors_11["change"] = donors_11["total_post_bill"] - donors_11["total_pre_bill"]
donors_11["pct_change"] = (donors_11["total_post_bill"] - donors_11["total_pre_bill"]) / donors_11["total_pre_bill"].abs()
donors_11.sort_values("total_post_bill", ascending=False).head()

,match_id,organizations,contributors,total_post_bill,total_pre_bill,pct_post_bill,pct_pre_bill,change,pct_change
27,U0000004022,[Thiel Capital],"[THIEL, PETER]","1,035,000.00","32,200.00",0.97,0.03,"1,002,800.00",31.14
11,U0000003260,"[Homemaker, Mt Vernon Investments, Winstar Farm]","[TROUTT, KENNETH A, TROUTT, KENNY A MR, TROUTT...","540,800.00","312,400.00",0.63,0.37,"228,400.00",0.73
22,U0000003690,"[U-Line Corp, Uline Inc]","[UIHLEIN, ELIZABETH, UIHLEIN, ELIZABETH MRS, U...","385,200.00","188,300.00",0.67,0.33,"196,900.00",1.05
33,U0000004393,"[Nevada Restaurant Services, Pere Antoine]","[ESTEY, ALLYSON, ESTEY, PATRICIA RIVERS MS]","102,400.00",0.00,1.00,0.00,"102,400.00",inf
2,Republican Governors Assn,[Republican Governors Assn],[REPUBLICAN GOVERNORS ASSOCIATION],"95,000.00","73,605.00",0.56,0.44,"21,395.00",0.29


#### How much did they give?

In [24]:
print("2011 post-bill total: $" + str(donors_11["total_post_bill"].sum()))
print("2011 pre-bill total: $" + str(donors_11["total_pre_bill"].sum()))

2011 post-bill total: $3241839.0
2011 pre-bill total: $7409350.0


### 2009

In [25]:
post_bill_donors_09 = post_bill_donors_committees_09.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
pre_bill_donors_09 = pre_bill_donors_committees_09.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()

In [26]:
donors_09 = post_bill_donors_09.merge(pre_bill_donors_09, how="outer", on="match_id", suffixes=["_post_bill", "_pre_bill"])
donors_09.drop(["organizations_pre_bill", "contributors_pre_bill"], axis=1, inplace=True)
donors_09.rename(columns={"organizations_post_bill": "organizations", "contributors_post_bill": "contributors"}, inplace=True)
donors_09["total_pre_bill"].fillna(0, inplace=True)
donors_09.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 88 entries, 0 to 87
Data columns (total 5 columns):
match_id           88 non-null object
organizations      50 non-null object
contributors       50 non-null object
total_post_bill    50 non-null float64
total_pre_bill     88 non-null float64
dtypes: float64(2), object(3)
memory usage: 4.1+ KB


In [27]:
donors_09["pct_post_bill"] = donors_09["total_post_bill"] / (donors_09["total_pre_bill"] + donors_09["total_post_bill"])
donors_09["pct_pre_bill"] = donors_09["total_pre_bill"] / (donors_09["total_pre_bill"] + donors_09["total_post_bill"])
donors_09["change"] = donors_09["total_post_bill"] - donors_09["total_pre_bill"]
donors_09["pct_change"] = (donors_09["total_post_bill"] - donors_09["total_pre_bill"]) / donors_09["total_pre_bill"].abs()
donors_09.sort_values("total_post_bill", ascending=False).head()

,match_id,organizations,contributors,total_post_bill,total_pre_bill,pct_post_bill,pct_pre_bill,change,pct_change
0,Chickasaw Nation,[Chickasaw Nation],"[CHICKASAW NATION, CHICKASAW NATION, THE, NATI...","75,600.00","96,700.00",0.44,0.56,"-21,100.00",-0.22
45,i3003697615,"[Mpi Research, Northwood Group]","[PARFET, BARBARA A MRS, PARFET, WILLIAM U, PAR...","51,400.00","1,000.00",0.98,0.02,"50,400.00",50.40
38,h1001217591,[Haworth Inc],"[HAWORTH, ETHELYN, HAWORTH, ETHELYN MRS, HAWOR...","40,000.00","2,400.00",0.94,0.06,"37,600.00",15.67
19,U0000003951,[Intercontinentalexchange Inc],"[LOEFFLER, KELLY L MS, SPRECHER, JEFFREY CRAIG...","37,400.00","19,910.00",0.65,0.35,"17,490.00",0.88
14,U0000003654,[Schweitzer Engineering Laboratories],"[SCHWEITZER, BEATRIZ, SCHWEITZER, EDMUND O III]","36,500.00","4,200.00",0.90,0.10,"32,300.00",7.69


#### How much did they give?

In [28]:
print("2009 post-bill total: $" + str(donors_09["total_post_bill"].sum()))
print("2009 pre-bill total: $" + str(donors_09["total_pre_bill"].sum()))

2009 post-bill total: $671846.0
2009 pre-bill total: $1949651.0


## How did each contributor's giving pattern change in each cycle compared with 2017?

### 2015

In [29]:
donors_15_17 = donors_17.merge(donors_15, how="outer", on="match_id", suffixes=["_17", "_15"])
donors_15_17.drop(["change_17", "pct_change_17", "change_15", "pct_change_15"],
            axis=1, inplace=True)
donors_15_17.rename(columns={"organizations_17": "organizations", "contributors_17": "contributors"}, inplace=True)
donors_15_17 = donors_15_17[["match_id", "contributors", "organizations", "total_pre_bill_15", "total_post_bill_15",
                 "pct_pre_bill_15", "pct_post_bill_15", "total_pre_bill_17", "total_post_bill_17",
                 "pct_pre_bill_17", "pct_post_bill_17"]]
donors_15_17.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158 entries, 0 to 157
Data columns (total 11 columns):
match_id              158 non-null object
contributors          158 non-null object
organizations         158 non-null object
total_pre_bill_15     111 non-null float64
total_post_bill_15    80 non-null float64
pct_pre_bill_15       80 non-null float64
pct_post_bill_15      80 non-null float64
total_pre_bill_17     158 non-null float64
total_post_bill_17    158 non-null float64
pct_pre_bill_17       158 non-null float64
pct_post_bill_17      158 non-null float64
dtypes: float64(8), object(3)
memory usage: 14.8+ KB


In [30]:
donors_15_17["giving_change"] = np.where((donors_15_17["pct_post_bill_17"] > donors_15_17["pct_post_bill_15"]) | (donors_15_17["pct_pre_bill_15"].isnull()), "Increased",
                                  np.where(donors_15_17["pct_post_bill_17"] < donors_15_17["pct_post_bill_15"], "Decreased",
                                           np.where(donors_15_17["pct_post_bill_17"] == donors_15_17["pct_post_bill_15"], "Stayed the same",
                                                   "Other")))
donors_15_17.head(1)

,match_id,contributors,organizations,total_pre_bill_15,total_post_bill_15,pct_pre_bill_15,pct_post_bill_15,total_pre_bill_17,total_post_bill_17,pct_pre_bill_17,pct_post_bill_17,giving_change
0,Air Line Pilots Assn,[AIR LINE PILOTS ASSOC INT'L POLITICAL ACTIO...,[Air Line Pilots Assn],nan,nan,nan,nan,"100,000.00","150,000.00",0.40,0.60,Increased


### 2013

In [31]:
donors_13_17 = donors_17.merge(donors_13, how="outer", on="match_id", suffixes=["_17", "_13"])
donors_13_17.drop(["change_17", "pct_change_17", "change_13", "pct_change_13"],
            axis=1, inplace=True)
donors_13_17.rename(columns={"organizations_17": "organizations", "contributors_17": "contributors"}, inplace=True)
donors_13_17 = donors_13_17[["match_id", "contributors", "organizations", "total_pre_bill_13", "total_post_bill_13",
                 "pct_pre_bill_13", "pct_post_bill_13", "total_pre_bill_17", "total_post_bill_17",
                 "pct_pre_bill_17", "pct_post_bill_17"]]
donors_13_17.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158 entries, 0 to 157
Data columns (total 11 columns):
match_id              158 non-null object
contributors          158 non-null object
organizations         158 non-null object
total_pre_bill_13     109 non-null float64
total_post_bill_13    62 non-null float64
pct_pre_bill_13       62 non-null float64
pct_post_bill_13      62 non-null float64
total_pre_bill_17     158 non-null float64
total_post_bill_17    158 non-null float64
pct_pre_bill_17       158 non-null float64
pct_post_bill_17      158 non-null float64
dtypes: float64(8), object(3)
memory usage: 14.8+ KB


In [32]:
donors_13_17["giving_change"] = np.where((donors_13_17["pct_post_bill_17"] > donors_13_17["pct_post_bill_13"]) | (donors_13_17["pct_pre_bill_13"].isnull()), "Increased",
                                  np.where(donors_13_17["pct_post_bill_17"] < donors_13_17["pct_post_bill_13"], "Decreased",
                                           np.where(donors_13_17["pct_post_bill_17"] == donors_13_17["pct_post_bill_13"], "Stayed the same",
                                                   "Other")))
donors_13_17.head(1)

,match_id,contributors,organizations,total_pre_bill_13,total_post_bill_13,pct_pre_bill_13,pct_post_bill_13,total_pre_bill_17,total_post_bill_17,pct_pre_bill_17,pct_post_bill_17,giving_change
0,Air Line Pilots Assn,[AIR LINE PILOTS ASSOC INT'L POLITICAL ACTIO...,[Air Line Pilots Assn],nan,nan,nan,nan,"100,000.00","150,000.00",0.40,0.60,Increased


### 2011

In [33]:
donors_11_17 = donors_17.merge(donors_11, how="outer", on="match_id", suffixes=["_17", "_11"])
donors_11_17.drop(["change_17", "pct_change_17", "change_11", "pct_change_11"],
            axis=1, inplace=True)
donors_11_17.rename(columns={"organizations_17": "organizations", "contributors_17": "contributors"}, inplace=True)
donors_11_17 = donors_11_17[["match_id", "contributors", "organizations", "total_pre_bill_11", "total_post_bill_11",
                 "pct_pre_bill_11", "pct_post_bill_11", "total_pre_bill_17", "total_post_bill_17",
                 "pct_pre_bill_17", "pct_post_bill_17"]]
donors_11_17.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158 entries, 0 to 157
Data columns (total 11 columns):
match_id              158 non-null object
contributors          158 non-null object
organizations         158 non-null object
total_pre_bill_11     106 non-null float64
total_post_bill_11    71 non-null float64
pct_pre_bill_11       71 non-null float64
pct_post_bill_11      71 non-null float64
total_pre_bill_17     158 non-null float64
total_post_bill_17    158 non-null float64
pct_pre_bill_17       158 non-null float64
pct_post_bill_17      158 non-null float64
dtypes: float64(8), object(3)
memory usage: 14.8+ KB


In [34]:
donors_11_17["giving_change"] = np.where((donors_11_17["pct_post_bill_17"] > donors_11_17["pct_post_bill_11"]) | (donors_11_17["pct_pre_bill_11"].isnull()), "Increased",
                                  np.where(donors_11_17["pct_post_bill_17"] < donors_11_17["pct_post_bill_11"], "Decreased",
                                           np.where(donors_11_17["pct_post_bill_17"] == donors_11_17["pct_post_bill_11"], "Stayed the same",
                                                   "Other")))
donors_11_17.head(1)

,match_id,contributors,organizations,total_pre_bill_11,total_post_bill_11,pct_pre_bill_11,pct_post_bill_11,total_pre_bill_17,total_post_bill_17,pct_pre_bill_17,pct_post_bill_17,giving_change
0,Air Line Pilots Assn,[AIR LINE PILOTS ASSOC INT'L POLITICAL ACTIO...,[Air Line Pilots Assn],"2,500.00",nan,nan,nan,"100,000.00","150,000.00",0.40,0.60,Increased


### 2009

In [35]:
donors_09_17 = donors_17.merge(donors_09, how="outer", on="match_id", suffixes=["_17", "_09"])
donors_09_17.drop(["change_17", "pct_change_17", "change_09", "pct_change_09"],
            axis=1, inplace=True)
donors_09_17.rename(columns={"organizations_17": "organizations", "contributors_17": "contributors"}, inplace=True)
donors_09_17 = donors_09_17[["match_id", "contributors", "organizations", "total_pre_bill_09", "total_post_bill_09",
                 "pct_pre_bill_09", "pct_post_bill_09", "total_pre_bill_17", "total_post_bill_17",
                 "pct_pre_bill_17", "pct_post_bill_17"]]
donors_09_17.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158 entries, 0 to 157
Data columns (total 11 columns):
match_id              158 non-null object
contributors          158 non-null object
organizations         158 non-null object
total_pre_bill_09     88 non-null float64
total_post_bill_09    50 non-null float64
pct_pre_bill_09       50 non-null float64
pct_post_bill_09      50 non-null float64
total_pre_bill_17     158 non-null float64
total_post_bill_17    158 non-null float64
pct_pre_bill_17       158 non-null float64
pct_post_bill_17      158 non-null float64
dtypes: float64(8), object(3)
memory usage: 14.8+ KB


In [36]:
donors_09_17["giving_change"] = np.where((donors_09_17["pct_post_bill_17"] > donors_09_17["pct_post_bill_09"]) | (donors_09_17["pct_pre_bill_09"].isnull()), "Increased",
                                  np.where(donors_09_17["pct_post_bill_17"] < donors_09_17["pct_post_bill_09"], "Decreased",
                                           np.where(donors_09_17["pct_post_bill_17"] == donors_09_17["pct_post_bill_09"], "Stayed the same",
                                                   "Other")))
donors_09_17.head(1)

,match_id,contributors,organizations,total_pre_bill_09,total_post_bill_09,pct_pre_bill_09,pct_post_bill_09,total_pre_bill_17,total_post_bill_17,pct_pre_bill_17,pct_post_bill_17,giving_change
0,Air Line Pilots Assn,[AIR LINE PILOTS ASSOC INT'L POLITICAL ACTIO...,[Air Line Pilots Assn],nan,nan,nan,nan,"100,000.00","150,000.00",0.40,0.60,Increased


## What proportion of donors increased the share of their annual giving that fell in the last two months of 2017 as compared with the same period in each of the previous years?

In [37]:
print(str(donors_15_17[donors_15_17["giving_change"] == "Increased"]["match_id"].count() / donors_15_17["match_id"].count() * 100) + "% of donors increased the share of their annual giving that fell in the last two months of the year between 2015 and 2017.")
print(str(donors_13_17[donors_13_17["giving_change"] == "Increased"]["match_id"].count() / donors_13_17["match_id"].count() * 100) + "% of donors increased the share of their annual giving that fell in the last two months of the year between 2013 and 2017.")
print(str(donors_11_17[donors_11_17["giving_change"] == "Increased"]["match_id"].count() / donors_11_17["match_id"].count() * 100) + "% of donors increased the share of their annual giving that fell in the last two months of the year between 2011 and 2017.")
print(str(donors_09_17[donors_09_17["giving_change"] == "Increased"]["match_id"].count() / donors_09_17["match_id"].count() * 100) + "% of donors increased the share of their annual giving that fell in the last two months of the year between 2009 and 2017.")

89.24050632911393% of donors increased the share of their annual giving that fell in the last two months of the year between 2015 and 2017.
83.54430379746836% of donors increased the share of their annual giving that fell in the last two months of the year between 2013 and 2017.
85.44303797468355% of donors increased the share of their annual giving that fell in the last two months of the year between 2011 and 2017.
85.44303797468355% of donors increased the share of their annual giving that fell in the last two months of the year between 2009 and 2017.


## And by how much did these individuals' giving increase?

In [38]:
print("These donors' giving in the last two months of the year increased by $" + str(donors_15_17[donors_15_17["giving_change"] == "Increased"]["total_post_bill_17"].sum() - donors_15_17[donors_15_17["giving_change"] == "Increased"]["total_post_bill_15"].sum()) + ", from $" + str(donors_15_17[donors_15_17["giving_change"] == "Increased"]["total_post_bill_15"].sum()) + " to $" + str(donors_15_17[donors_15_17["giving_change"] == "Increased"]["total_post_bill_17"].sum()) + " between 2015 and 2017.")
print("These donors' giving in the last two months of the year increased by $" + str(donors_13_17[donors_13_17["giving_change"] == "Increased"]["total_post_bill_17"].sum() - donors_13_17[donors_13_17["giving_change"] == "Increased"]["total_post_bill_13"].sum()) + ", from $" + str(donors_13_17[donors_13_17["giving_change"] == "Increased"]["total_post_bill_13"].sum()) + " to $" + str(donors_13_17[donors_13_17["giving_change"] == "Increased"]["total_post_bill_17"].sum()) + " between 2013 and 2017.")
print("These donors' giving in the last two months of the year increased by $" + str(donors_11_17[donors_11_17["giving_change"] == "Increased"]["total_post_bill_17"].sum() - donors_11_17[donors_11_17["giving_change"] == "Increased"]["total_post_bill_11"].sum()) + ", from $" + str(donors_11_17[donors_11_17["giving_change"] == "Increased"]["total_post_bill_11"].sum()) + " to $" + str(donors_11_17[donors_11_17["giving_change"] == "Increased"]["total_post_bill_17"].sum()) + " between 2011 and 2017.")
print("These donors' giving in the last two months of the year increased by $" + str(donors_09_17[donors_09_17["giving_change"] == "Increased"]["total_post_bill_17"].sum() - donors_09_17[donors_09_17["giving_change"] == "Increased"]["total_post_bill_09"].sum()) + ", from $" + str(donors_09_17[donors_09_17["giving_change"] == "Increased"]["total_post_bill_09"].sum()) + " to $" + str(donors_09_17[donors_09_17["giving_change"] == "Increased"]["total_post_bill_17"].sum()) + " between 2009 and 2017.")

These donors' giving in the last two months of the year increased by $35512029.0, from $-6589136.0 to $28922893.0 between 2015 and 2017.
These donors' giving in the last two months of the year increased by $21880518.0, from $1155975.0 to $23036493.0 between 2013 and 2017.
These donors' giving in the last two months of the year increased by $24340851.0, from $613592.0 to $24954443.0 between 2011 and 2017.
These donors' giving in the last two months of the year increased by $31881913.0, from $267350.0 to $32149263.0 between 2009 and 2017.


## How many donors in each year gave all of their money in the last two months of each year?

In [39]:
print("In 2017, " + str(donors_17[donors_17["pct_post_bill"] == 1]["match_id"].count()) + " donors gave all their money in the last two months.")
print("In 2015, " + str(donors_15[donors_15["pct_post_bill"] == 1]["match_id"].count()) + " donors gave all their money in the last two months.")
print("In 2013, " + str(donors_13[donors_13["pct_post_bill"] == 1]["match_id"].count()) + " donors gave all their money in the last two months.")
print("In 2011, " + str(donors_11[donors_11["pct_post_bill"] == 1]["match_id"].count()) + " donors gave all their money in the last two months.")
print("In 2009, " + str(donors_09[donors_09["pct_post_bill"] == 1]["match_id"].count()) + " donors gave all their money in the last two months.")

In 2017, 44 donors gave all their money in the last two months.
In 2015, 2 donors gave all their money in the last two months.
In 2013, 4 donors gave all their money in the last two months.
In 2011, 3 donors gave all their money in the last two months.
In 2009, 7 donors gave all their money in the last two months.


# What does this giving look like on a contribution-by-contribution level?

In [40]:
def run_contributions_query(prefix, suffix, reference_cycle, reference_start, reference_end, cycle, start, end):
    query = """CREATE MATERIALIZED VIEW IF NOT EXISTS """ + prefix + """_contributions_""" + suffix + """ AS
    SELECT fectransid,
    match_id,
    organization,
    contributor, date, sum(SUM),
    cmte_id,
    pacshort AS committee
FROM
  (SELECT fectransid,
          match_id,
          orgname AS organization,
          contrib AS contributor, date, sum(amount),
                                        cmteid AS cmte_id
   FROM
     (SELECT CASE
                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                 ELSE orgname
             END AS match_id
      FROM crp_contributions
      LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
      AND crp_committees.cycle = '""" + reference_cycle + """'
      WHERE primcode IN ('J1100',
                         'J2200',
                         'J2400',
                         'Z1100',
                         'Z4100',
                         'Z4500',
                         'Z5100')
                            AND date >= '""" + reference_start + """'
                            AND date <= '""" + reference_end + """'
                            AND crp_contributions.cycle = '""" + reference_cycle + """'
        AND TYPE NOT IN ('11J',
                         '15C',
                         '15E',
                         '15I',
                         '15J',
                         '15T',
                         '18J',
                         '19J',
                         '30J',
                         '30F',
                         '31J',
                         '31F',
                         '32J',
                         '32F')
      GROUP BY match_id
      HAVING sum(amount) >= 50000) AS gop_donors
   JOIN crp_contributions ON CASE
                                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                                 ELSE crp_contributions.orgname
                             END = gop_donors.match_id
                       WHERE date >= '""" + start + """'
                         AND date <= '""" + end + """'
                         AND CYCLE = '""" + cycle + """'
     AND TYPE NOT IN ('11J',
                      '15C',
                      '15E',
                      '15I',
                      '15J',
                      '15T',
                      '18J',
                      '19J',
                      '30J',
                      '30F',
                      '31J',
                      '31F',
                      '32J',
                      '32F')
   GROUP BY fectransid,
            match_id,
            organization,
            contributor, date, cmte_id) AS donors_committees
JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
                    AND crp_committees.CYCLE = '""" + cycle + """'
WHERE primcode IN ('J1100',
                   'J2200',
                   'J2400',
                   'Z1100',
                   'Z4100',
                   'Z4500',
                   'Z5100')
GROUP BY fectransid,
         match_id,
         organization,
         contributor, date, cmte_id,
                            committee;
                            SELECT *
                            FROM """ + prefix + """_contributions_"""+ suffix +""";"""
    return pd.read_sql(query, con=conn)

## Return the data for each cycle.

In [41]:
post_bill_contributions_17 = run_contributions_query("post_bill", "17", "2018", "2017-11-02", "2017-12-31", "2018", "2017-11-02", "2017-12-31")
pre_bill_contributions_17 = run_contributions_query("pre_bill", "17", "2018", "2017-11-02", "2017-12-31", "2018", "2017-01-01", "2017-11-01")

In [42]:
post_bill_contributions_15 = run_contributions_query("post_bill", "15", "2018", "2017-11-02", "2017-12-31", "2016", "2015-11-02", "2015-12-31")
pre_bill_contributions_15 = run_contributions_query("pre_bill", "15", "2018", "2017-11-02", "2017-12-31", "2016", "2015-01-01", "2015-11-01")

In [43]:
post_bill_contributions_13 = run_contributions_query("post_bill", "13", "2018", "2017-11-02", "2017-12-31", "2014", "2013-11-02", "2013-12-31")
pre_bill_contributions_13 = run_contributions_query("pre_bill", "13", "2018", "2017-11-02", "2017-12-31", "2014", "2013-01-01", "2013-11-01")

In [44]:
post_bill_contributions_11 = run_contributions_query("post_bill", "11", "2018", "2017-11-02", "2017-12-31", "2012", "2011-11-02", "2011-12-31")
pre_bill_contributions_11 = run_contributions_query("pre_bill", "11", "2018", "2017-11-02", "2017-12-31", "2012", "2011-01-01", "2011-11-01")

In [45]:
post_bill_contributions_09 = run_contributions_query("post_bill", "09", "2018", "2017-11-02", "2017-12-31", "2010", "2009-11-02", "2009-12-31")
pre_bill_contributions_09 = run_contributions_query("pre_bill", "09", "2018", "2017-11-02", "2017-12-31", "2010", "2009-01-01", "2009-11-01")

## Export data to Excel.

In [46]:
writer = pd.ExcelWriter("data/megadonors.xlsx")
donors_15_17.to_excel(writer, "donors_15_17", startcol=0, index=False)
donors_13_17.to_excel(writer, "donors_13_17", startcol=0, index=False)
donors_11_17.to_excel(writer, "donors_11_17", startcol=0, index=False)
donors_09_17.to_excel(writer, "donors_09_17", startcol=0, index=False)
post_bill_donors_committees_17.to_excel(writer, "post_bill_donors_committees_17", startcol=0, index=False)
pre_bill_donors_committees_17.to_excel(writer, "pre_bill_donors_committees_17", startcol=0, index=False)
post_bill_donors_committees_15.to_excel(writer, "post_bill_donors_committees_15", startcol=0, index=False)
pre_bill_donors_committees_15.to_excel(writer, "pre_bill_donors_committees_15", startcol=0, index=False)
post_bill_donors_committees_13.to_excel(writer, "post_bill_donors_committees_13", startcol=0, index=False)
pre_bill_donors_committees_13.to_excel(writer, "pre_bill_donors_committees_13", startcol=0, index=False)
post_bill_donors_committees_11.to_excel(writer, "post_bill_donors_committees_11", startcol=0, index=False)
pre_bill_donors_committees_11.to_excel(writer, "pre_bill_donors_committees_11", startcol=0, index=False)
post_bill_donors_committees_09.to_excel(writer, "post_bill_donors_committees_09", startcol=0, index=False)
pre_bill_donors_committees_09.to_excel(writer, "pre_bill_donors_committees_09", startcol=0, index=False)
post_bill_contributions_17.to_excel(writer, "post_bill_contributions_17", startcol=0, index=False)
pre_bill_contributions_17.to_excel(writer, "pre_bill_contributions_17", startcol=0, index=False)
post_bill_contributions_15.to_excel(writer, "post_bill_contributions_15", startcol=0, index=False)
pre_bill_contributions_15.to_excel(writer, "pre_bill_contributions_15", startcol=0, index=False)
post_bill_contributions_13.to_excel(writer, "post_bill_contributions_13", startcol=0, index=False)
pre_bill_contributions_13.to_excel(writer, "pre_bill_contributions_13", startcol=0, index=False)
post_bill_contributions_11.to_excel(writer, "post_bill_contributions_11", startcol=0, index=False)
pre_bill_contributions_11.to_excel(writer, "pre_bill_contributions_11", startcol=0, index=False)
post_bill_contributions_09.to_excel(writer, "post_bill_contributions_09", startcol=0, index=False)
pre_bill_contributions_09.to_excel(writer, "pre_bill_contributions_09", startcol=0, index=False)
writer.save()